<a href="https://colab.research.google.com/github/sayonaramagalhaes/Trabalho-Final---ML/blob/main/Feature_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torchvision import models, transforms
from PIL import Image
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

#Baixar o dataset
path = kagglehub.dataset_download("shaunthesheep/microsoft-catsvsdogs-dataset")

print("Path to dataset files:", path)

#Verificar o diretório

print("Conteúdo do diretório do dataset:", os.listdir(path))

#Definir o pré-processamento das imagens
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensiona para 224x224
    transforms.Grayscale(num_output_channels=3),  # Garante que a imagem tenha 3 canais (RGB)
    transforms.ToTensor(),  # Converte para tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normaliza com os valores do ImageNet
])

# Carregar o modelo AlexNet pré-treinado
model = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)

# Remover a última camada (classificador)
model.classifier = torch.nn.Sequential(*list(model.classifier.children())[:-1])

#avaliação
model.eval()

# Função  extrair as características
def extract_features(image_path):
    try:
        # Abrir a imagem e aplicar as transformações
        image = Image.open(image_path).convert('RGB')  # Garantir que a imagem esteja em RGB
        image = transform(image).unsqueeze(0)  # Adiciona uma dimensão extra para o batch

        # Passar a imagem pela rede (sem a última camada)
        with torch.no_grad():  # Desativa o cálculo de gradientes para economizar memória
            features = model.features(image)  # Passa pela parte convolucional
            features = features.view(features.size(0), -1)  # Achata as características para um vetor unidimensional

        return features
    except (PIL.UnidentifiedImageError, OSError) as e:
        print(f"Erro ao processar imagem {image_path}: {e}")
        return None


# Corrigir caminho para as imagens
dataset_path = os.path.join(path, "PetImages")

# Verifique a estrutura do diretório novamente
print("Estrutura do diretório de imagens:", os.listdir(dataset_path))

# Obter subpastas (ex: 'cats' e 'dogs')
subfolders = os.listdir(dataset_path)
print("Subpastas do dataset:", subfolders)

#Listas para armazenar características e rótulos
features_list = []
labels_list = []

# Iterar sobre as subpastas e extrair as características
for subfolder in subfolders:
    subfolder_path = os.path.join(dataset_path, subfolder)
    if os.path.isdir(subfolder_path):
        images = os.listdir(subfolder_path)

        for image_name in images:
            image_path = os.path.join(subfolder_path, image_name)
            # Verifica se é um arquivo de imagem, com base na extensão do arquivo
            if image_name.endswith(('.jpg', '.jpeg', '.png')):
                features = extract_features(image_path)  # Extrai as características
                if features is not None:
                    features_list.append(features.flatten().numpy())  # Armazena as características como vetor
                    labels_list.append(0 if subfolder == 'Cat' else 1)  # 0 gatos, 1 cachorros

#Converter listas para arrays numpy
X = np.array(features_list)
y = np.array(labels_list)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Treinar o modelo de Regressão Logística
logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(X_train, y_train)

#Fazer previsões com o modelo treinado
y_pred = logreg_model.predict(X_test)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Acurácia do modelo: {accuracy:.4f}")
print(f"F1-Score do modelo: {f1:.4f}")



In [ ]:
#precisao
from sklearn.metrics import classification_report

# Fazer previsões com o modelo treinado
y_pred = logreg_model.predict(X_test)

# Avaliar o modelo
report = classification_report(y_test, y_pred)

print("Relatório de Classificação:\n", report)



Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.93      0.94      0.94      2485
           1       0.94      0.93      0.94      2515

    accuracy                           0.94      5000
   macro avg       0.94      0.94      0.94      5000
weighted avg       0.94      0.94      0.94      5000

